In [103]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder 

## read files

In [104]:
df_lst = []
with open(r'C:\Users\braul\OneDrive\Desktop\GF_etsy_scrap\item_attr.txt') as f:
    for i in f:
        #print(i)
        dictionary = json.loads(i)
        #print(dictionary['item_id'])
        frame = pd.DataFrame.from_dict(dictionary)
        #print(len(frame))
        df_lst.append(frame)

df = pd.concat(df_lst)
df.head()


,date,order,item_id,page_num,iteration,position_num,shop_id,item_link,target,title,...,stock,offer_count,lowPrice,highPrice,date_listed,price_dpv,num_img,add_url,tags,num_fav
0,2023-07-16,1,1108461163,1,1,1,32322470,https://www.etsy.com/de-en/listing/1108461163/...,etsy.1108461163,Cherry Wood Bookmark Flying Turtle,...,None,13,15.00,20.00,"15 Jun, 2023",None,6,https://www.etsy.com/de-en/search?ga_order=mos...,Leaf,71
1,2023-07-16,1,1108461163,1,1,1,32322470,https://www.etsy.com/de-en/listing/1108461163/...,etsy.1108461163,Cherry Wood Bookmark Flying Turtle,...,None,13,15.00,20.00,"15 Jun, 2023",None,6,https://www.etsy.com/de-en/search?ga_order=mos...,Turtle,71
2,2023-07-16,1,1108461163,1,1,1,32322470,https://www.etsy.com/de-en/listing/1108461163/...,etsy.1108461163,Cherry Wood Bookmark Flying Turtle,...,None,13,15.00,20.00,"15 Jun, 2023",None,6,https://www.etsy.com/de-en/search?ga_order=mos...,Engraving,71
3,2023-07-16,1,1108461163,1,1,1,32322470,https://www.etsy.com/de-en/listing/1108461163/...,etsy.1108461163,Cherry Wood Bookmark Flying Turtle,...,None,13,15.00,20.00,"15 Jun, 2023",None,6,https://www.etsy.com/de-en/search?ga_order=mos...,None,71
4,2023-07-16,1,1108461163,1,1,1,32322470,https://www.etsy.com/de-en/listing/1108461163/...,etsy.1108461163,Cherry Wood Bookmark Flying Turtle,...,None,13,15.00,20.00,"15 Jun, 2023",None,6,https://www.etsy.com/de-en/search?ga_order=mos...,Clouds,71


### Etsy adds listings

In [69]:
df['is_add'] = [True if x.find('plkey') > 0 else False for x in df['add_url']]
df['is_add'].unique()

array([ True, False])

### Correct data types

In [82]:
df['page_num'] = df['page_num'].astype('int64')
df['position_num'] = df['position_num'].astype('int64')
df.dtypes

date             object
order             int64
item_id          object
page_num          int64
iteration         int64
position_num      int64
shop_id          object
item_link        object
target           object
title            object
img_text         object
video            object
rating           object
shop_reviews     object
price_from       object
price            object
seller_batch     object
discount         object
free_delivery    object
description      object
rating_value     object
review_count     object
stock            object
offer_count      object
lowPrice         object
highPrice        object
date_listed      object
price_dpv        object
num_img           int64
add_url          object
tags             object
num_fav          object
is_add             bool
dtype: object

### Relative position

In [83]:
df['relative_position'] = df.apply(lambda x: 64*(x.page_num/2)+x.position_num,axis=1)

### Stadistics

In [84]:
len_df = len(df)
unique_item_id = df['item_id'].nunique()
unique_shop_id = df['shop_id'].nunique()
print('number of rows: {}'.format(len_df))
print('unique items: {}'.format(unique_item_id))
print('unique shop: {}'.format(unique_shop_id))

number of rows: 7704
unique items: 510
unique shop: 345


In [85]:
df.describe()

,order,page_num,iteration,position_num,num_img,relative_position
count,7704.000000,7704.000000,7704.000000,7704.000000,7704.000000,7704.000000
mean,32.687435,5.489486,5.489486,185.116822,7.029595,360.780374
std,18.416459,2.889437,2.889437,156.511092,2.523452,229.365528
min,1.000000,1.000000,1.000000,1.000000,1.000000,33.000000
25%,17.000000,3.000000,3.000000,16.000000,5.000000,157.000000
50%,32.000000,5.000000,5.000000,163.000000,7.000000,307.000000
75%,49.000000,8.000000,8.000000,323.000000,10.000000,547.000000
max,64.000000,10.000000,10.000000,480.000000,10.000000,800.000000


### Tags

In [91]:
labelencoder= LabelEncoder()
df['tags_encode'] = labelencoder.fit_transform(df['tags']) #fitting and transforming the desired categorical column.
df['tags_encode'].unique()

array([ 729, 1061,  481, ...,  345,  358,  987])

In [97]:
df.columns

Index(['date', 'order', 'item_id', 'page_num', 'iteration', 'position_num',
       'shop_id', 'item_link', 'target', 'title', 'img_text', 'video',
       'rating', 'shop_reviews', 'price_from', 'price', 'seller_batch',
       'discount', 'free_delivery', 'description', 'rating_value',
       'review_count', 'stock', 'offer_count', 'lowPrice', 'highPrice',
       'date_listed', 'price_dpv', 'num_img', 'add_url', 'tags', 'num_fav',
       'is_add', 'relative_position', 'tags_encode'],
      dtype='object')

In [100]:
df_tags = df[['item_id','page_num','relative_position','tags']]
df_tags = df_tags.groupby(['item_id','page_num','relative_position'],as_index=False)['tags'].count()
df_tags = df_tags[['tags','relative_position']]
df_tags.corr()

,tags,relative_position
tags,1.000000,-0.047378
relative_position,-0.047378,1.000000


In [102]:
df_tags = df[['relative_position','tags']]
df_tags.apply(lambda x: x.factorize()[0]).corr()

,relative_position,tags
relative_position,1.000000,0.446699
tags,0.446699,1.000000


In [88]:
df['tags'].unique()

array(['Leaf', 'Turtle', 'Engraving', ..., "Children's Birthday Gift",
       'Christmas Gift Girls', 'School Start Gift'], dtype=object)

In [90]:
#most used tags in the 10 first pages
df.groupby("tags",as_index=False)['order'].count().sort_values(by='order',ascending=False).head(10)

,tags,order
1324,bookmark,275
155,Bookmark,93
551,Gift,84
283,Books,61
1265,book accessories,56
1854,gift idea,54
1284,book lover,51
1424,bookworm,51
1944,handmade,40
1389,bookmarks,38


In [60]:
df.columns

Index(['date', 'order', 'item_id', 'page_num', 'position_num', 'shop_id',
       'item_link', 'target', 'title', 'img_text', 'video', 'rating',
       'shop_reviews', 'price_from', 'price', 'seller_batch', 'discount',
       'free_delivery', 'description', 'rating_value', 'review_count', 'stock',
       'offer_count', 'lowPrice', 'highPrice', 'price_dpv', 'num_img',
       'add_url', 'tags', 'num_fav', 'is_add', 'relative_position',
       'tags_encode'],
      dtype='object')

In [61]:
subset = ['date','order', 'item_id', 'page_num', 'position_num', 'shop_id', 'item_link',
       'target', 'title', 'img_text', 'video', 'rating', 'shop_reviews',
       'price_from', 'price', 'seller_batch', 'discount', 'free_delivery',
       'description', 'rating_value', 'review_count', 'stock', 'offer_count',
       'lowPrice', 'highPrice', 'price_dpv', 'num_img','is_add','num_fav']
df_drop_duplications = df.drop_duplicates(subset=subset)

In [62]:
df_drop_duplications.dtypes

date                  object
order                  int64
item_id               object
page_num               int64
position_num           int64
shop_id               object
item_link             object
target                object
title                 object
img_text              object
video                 object
rating                object
shop_reviews          object
price_from            object
price                 object
seller_batch          object
discount              object
free_delivery         object
description           object
rating_value          object
review_count          object
stock                 object
offer_count           object
lowPrice              object
highPrice             object
price_dpv             object
num_img                int64
add_url               object
tags                  object
num_fav               object
is_add                  bool
relative_position    float64
tags_encode            int32
dtype: object

In [63]:
len(df_drop_duplications)

61

In [64]:
add_listings = len(df_drop_duplications[df_drop_duplications['is_add']==True])
total_listings = len(df_drop_duplications)
print('{}% are add listings'.format(add_listings/total_listings*100))

22.950819672131146% are add listings


In [65]:
df_drop_duplications['price'] = df_drop_duplications['price'].astype('float').copy()
df_drop_duplications.dtypes

C:\Users\braul\AppData\Local\Temp\ipykernel_32792\1149374823.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop_duplications['price'] = df_drop_duplications['price'].astype('float').copy()


date                  object
order                  int64
item_id               object
page_num               int64
position_num           int64
shop_id               object
item_link             object
target                object
title                 object
img_text              object
video                 object
rating                object
shop_reviews          object
price_from            object
price                float64
seller_batch          object
discount              object
free_delivery         object
description           object
rating_value          object
review_count          object
stock                 object
offer_count           object
lowPrice              object
highPrice             object
price_dpv             object
num_img                int64
add_url               object
tags                  object
num_fav               object
is_add                  bool
relative_position    float64
tags_encode            int32
dtype: object

In [66]:
df_drop_duplications['price'].mean()

19.206229508196724

In [55]:
items_w_vide = df_drop_duplications.groupby('video',as_index=False)['item_id'].count().iloc[0,1]
items_w_vide/len(df_drop_duplications)
#df_drop_duplications['video'].unique()

0.40625

In [63]:
df_title = df_drop_duplications['title']
df_title['split'] = [x.split(' ') for x in df_drop_duplications['title']]
df_title['split']

C:\Users\braul\AppData\Local\Temp\ipykernel_13056\3075815135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_title['split'] = [x.split(' ') for x in df_drop_duplications['title']]
C:\Users\braul\AppData\Local\Temp\ipykernel_13056\3075815135.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_title['split'] = [x.split(' ') for x in df_drop_duplications['title']]


[['Cherry', 'Wood', 'Bookmark', 'Bookworm'],
 ['Cute',
  'bookmark',
  'custom',
  'bookmark',
  'metal',
  'bookmark',
  'bookmark',
  'bookmark',
  'dragon',
  'jade',
  'Celtic',
  'jewel',
  'handmade',
  'handmade',
  'gift',
  'present'],
 ['Genuine',
  'Full',
  'Grain',
  'Leather',
  'Bookmark',
  'Custom',
  'Name',
  'Bookmark',
  'Personalized',
  'Gift',
  'Bookmark',
  'Cute',
  'Bookmarks',
  'For',
  'Women',
  'Aesthetic',
  'Bookmark',
  'For',
  'Her'],
 ['LITTLE',
  'LEADERS',
  'Boys',
  'BOOKMARKS',
  'Diverse',
  'Books',
  'Boy',
  'bookmark',
  'Readers',
  'Gifts',
  'Party',
  'favours',
  'Black',
  'Boy',
  'Joy',
  'Empowering',
  'Boy',
  'Brown',
  'Boys'],
 ['F',
  'ck',
  'Off',
  'I',
  'm',
  'Reading',
  'Pink',
  'Recycled',
  'Paper',
  'Bookmark'],
 ['bookmark',
  'colored',
  'Resin',
  'bookmark',
  'Bookmark',
  'Personalized',
  'personalized',
  'gift'],
 ['Bookmark',
  'Set',
  'ASTROLOGY',
  '4',
  'Bookmarks',
  'Moon',
  'and',
  'Stars'

In [74]:
title_word_list = []
for x in df_title['split']:
    for y in x:
        title_word_list.append(y.lower())
df_title_word = pd.DataFrame({'words':title_word_list})
df_title_word.groupby('words',as_index=False).size().sort_values(by='size',ascending=False).head(10)

,words,size
35,bookmark,99
36,bookmarks,38
122,gift,34
216,personalized,17
298,with,16
112,for,16
241,resin,14
161,leather,12
72,custom,10
131,handmade,10


In [73]:
df_description = df_drop_duplications['description']
df_description['split'] = [x.split(' ') for x in df_drop_duplications['description']]
df_description['split']

C:\Users\braul\AppData\Local\Temp\ipykernel_13056\3502312123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_description['split'] = [x.split(' ') for x in df_drop_duplications['description']]
C:\Users\braul\AppData\Local\Temp\ipykernel_13056\3502312123.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_description['split'] = [x.split(' ') for x in df_drop_duplications['description']]


[['Cherry',
  'wood',
  'bookmark',
  'with',
  'a',
  'detailed',
  'engraving',
  'of',
  'a',
  'illustration',
  'based',
  'on',
  'a',
  'selfmade',
  'drawing',
  'with',
  'a',
  'leather',
  'lace',
  'hamp',
  'as',
  'vegan',
  'option',
  'if',
  'preferred',
  'The',
  'size',
  'of',
  'the',
  'bookmark',
  'is',
  'approximately',
  '5',
  '7',
  'quot',
  'x1',
  '57',
  'quot',
  'x0',
  '08',
  'quot',
  'Every',
  'engraving',
  'is',
  'based',
  'on',
  'a',
  'selfmade',
  'traditional',
  'or',
  'digital',
  'drawing',
  'of',
  'a',
  'original',
  'made',
  'or',
  'copyright',
  'free',
  'character',
  'Except',
  'the',
  'packaging',
  'tape',
  'which',
  'is',
  'used',
  'every',
  'economically',
  'every',
  'oder',
  'is',
  'packed',
  'with',
  '100',
  'recycled',
  'and',
  'again',
  'recyclable',
  'materials',
  'Copyright',
  '2021',
  'WoodwayIllustrations',
  'by',
  'Guido',
  'Heinrich'],
 ['Embellish',
  'your',
  'reading',
  'with',
 

In [80]:
description_word_list = []
for x in df_description['split']:
    for y in x:
        if len(y) > 5:
            description_word_list.append(y.lower())
        else:
            pass
df_description_word = pd.DataFrame({'words':description_word_list})
df_description_word.groupby('words',as_index=False).size().sort_values(by='size',ascending=False).head(20)

,words,size
113,bookmark,148
114,bookmarks,98
736,please,75
918,shipping,41
711,perfect,40
815,reading,29
463,handmade,29
555,leather,28
190,colors,25
1062,unique,25


In [85]:
df['stock'].mean()

266.8982387475538

In [88]:
df['stock'].mean()

array([None, 3, 21, 45, 64, 217, 11, 6, 649, 63, 100, 1, 911, 7, 4, 994,
       36, 10, 20, 72, 12, 9, 15, 2, 4976, 224, 93, 18, 5, 29, 25, 960,
       38], dtype=object)

In [18]:
df['etsy_add'] = [ x.find('plkey=') for x in df['item_link'] ]

In [21]:
df['etsy_add']

0    -1
1    -1
2    -1
3    -1
4    -1
     ..
9    -1
10   -1
11   -1
12   -1
13   -1
Name: etsy_add, Length: 813, dtype: int64

In [24]:
x = 'https://www.etsy.com/de/listing/1309259435/lesezeichplkey=en-personalisierte-aus-resin?ga_order=most_relevant&ga_search_type=all&ga_view_type=gallery&ga_search_query=bookmarks&ref=sr_gallery-2-10&organic_search_click=1'
ans = x.find('plkey=')
print(ans)

52
